In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import glob
import torch.nn as nn
import os.path as osp
import torchvision
import matplotlib.pyplot as plt
import torch
#import rotateImage
import cv2
import numpy as np
#import split_folders
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
from random import choice
from collections import OrderedDict

In [ ]:
model = torch.load('/content/carPartModel-305.pth')
for names in model:
  print(names)
# for name,param in model.items():
#   if name=='state_dict':
#     print(param)

In [ ]:
for name, param in model.named_parameters():
    print(name, ':', param.requires_grad)

In [ ]:
def load_model(checkpoint_path):
  chpt = torch.load(checkpoint_path)
  net = torchvision.models.resnet50(pretrained=True)
  fc = nn.Sequential(OrderedDict([
    ("dropout", nn.Dropout(0.4)),
    ("out", nn.Linear(2048, 11))]))
  net.fc = fc

  for param, x in net.named_parameters():
    if param.startswith("fc.out"):
        x.requires_grad = True
    else:
        x.requires_grad = False

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  #net.class_to_idx = classes
  net = net.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    
  net.class_to_idx = chpt['class_to_idx']
    
     
    
  net.load_state_dict(chpt['state_dict'])
    
  return net

In [ ]:
the_model = load_model('/content/carPartModel-305.pth')

In [ ]:
for name, param in the_model.named_parameters():
    print(name, ':', param.requires_grad)

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224, device='cuda')
onnx_path =  "/content/pytorch_ONNx.onnx"

torch.onnx.export(the_model, dummy_input, onnx_path, verbose=False)
print('Model exported to ' + onnx_path)